# LangChainを使ってChatGPTに独自データに関する質問に回答してもらう

このサンプルでは、インターネット上からダウンロードしたPDFファイルに関する質問に対しての回答をしてもらいます。
PDFファイルはインターネットからダウンロードします。
URLを変更するなどすることで、独自のPDFファイルを代わりに読み込ませることも可能です。


以下に処理の概要を示します。

0.   OpenAPIキーの設定と、関連ライブラリのインストールとインポート
1.   PDFをロードし、LangChainを使ってチャンクに分割する
2.   テキスト情報を数値化（Embedding)し、保存する
3.   データ取得関数を作成する。
4.   チャットbotの作成

こちらの記事は、Liam Ottleyさんの記事を参考にして独自の情報を追加しています。  
動画は以下のYoutubeからご参照ください。
  [YouTube](https://youtube.com/@LiamOttley)





# 0. OpenAPIキーの設定と、関連ライブラリのインストール
利用するライブラリをpipを使ってインストールします。

In [9]:
# RUN THIS CELL FIRST!//
!pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu requests beautifulsoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
import os
import pandas as pd
import requests
import textract
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


以下にOpenAIから取得したキーを設定します。キーは以下のURLから取得することができます。  
（アカウントの作成が必要です）  
https://platform.openai.com/account/api-keys



In [11]:
# ここにOpenAIから取得したキーを設定します。
os.environ["OPENAI_API_KEY"] = ''

インターネット上からPDFファイル（日本語で書かれたマニュアル）をダウンロードして、ローカルファイルとして保存します。

# 1. データをダウンロードし、LangChainを使ってチャンクに分割する
ここでは、自社の情報と仮定して、2つのPDFファイルを読み込みます。必要に応じて読み込むファイルを自社のマニュアルなどに置き換えてみてください。




### ドキュメントのロード＆チャンク分割



In [12]:
# Step 1: Convert PDF to text
doc = textract.process("rules.pdf")
# Step 2: Save to .txt and reopen (helps prevent issues)
with open('attention_is_all_you_need.txt', 'w') as f:
    f.write(doc.decode('utf-8'))

with open('attention_is_all_you_need.txt', 'r') as f:
    text = f.read()

# Step 3: Create function to count tokens
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# Step 4: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Token indices sequence length is longer than the specified maximum sequence length for this model (1944 > 1024). Running this sequence through the model will result in indexing errors


# 2. 読み込んだデータvector storeに保存する
ここでは、これまでに読み込んだ3つの情報をVector storeに保存します。。
Vector storeはいくつも種類がありますが、今回はFacebook製のFAISSを利用します。  


> 利用可能なVectorStoreはこちらを参照してください。
https://python.langchain.com/en/latest/modules/indexes/vectorstores.html#




In [13]:
# Get embedding model
embeddings = OpenAIEmbeddings()

#  vector databaseの作成
db = FAISS.from_documents(chunks, embeddings)

# 3. データ取得関数のセットアップ

## ユーザからのクエリを使って関連するデータを取得できる様にQAチェインを作成する

ここで、これまでにロードしたドキュメントに書かれた情報に関する質問を投げてみて、期待する結果が返ってくるかどうかを確認します。


>  以下コード内のtemperatureを変更することにより、情報の精度を上げることができます。(0-2までの値で指定）0にした場合、質問内容の回答がはっきりわからない場合はI don't knowと言われます。。



In [14]:
# 得られた情報から回答を導き出すためのプロセスを以下の4つから選択する。いずれもProsとConsがあるため、適切なものを選択する必要がある。
# staffing ... 得られた候補をそのままインプットとする
# map_reduce ... 得られた候補のサマリをそれぞれ生成し、そのサマリのサマリを作ってインプットとする
# map_rerank ... 得られた候補にそれぞれスコアを振って、いちばん高いものをインプットとして回答を得る
# refine  ... 得られた候補のサマリを生成し、次にそのサマリと次の候補の様裏を作ることを繰り返す
chain = load_qa_chain(OpenAI(temperature=0.2,max_tokens=1000), chain_type="stuff")
query = "how to play"
docs = db.similarity_search(query)
# langchainを使って検索
chain.run(input_documents=docs, question=query)

' You can visit plaidhatgames.com and watch video explanations.'

# 4. チャットbotを作成する（Option)
ここでは、これまでに読み込んだドキュメントを使って簡易のチャットボットを作成します。
ここを実行するとダイアログが表示されますので、前に読み込ませたドキュメントに関する質問をして見てください。

In [15]:
from IPython.display import display
import ipywidgets as widgets

# vextordbをretrieverとして使うconversation chainを作成します。これはチャット履歴の管理も可能にします。
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

In [16]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""
    
    if query.lower() == 'exit':
        print("Thank you for using the State of the Union chatbot!")
        return
    
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    
    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Transformers chatbot! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Welcome to the Transformers chatbot! Type 'exit' to stop.


/var/folders/96/jfcsm87117x90kt7g3gnt3pw0000gp/T/ipykernel_86865/3970211031.py:20: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_submit)


Text(value='', placeholder='Please enter your question:')